In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
#from nltk.tag import pos_tag
#from nltk.chunk import ne_chunk
#from nltk.corpus import stopwords

df = pd.read_csv('onion-or-not.csv')
y = df['label']

In [ ]:
# see shape and contents of dataframe
df

In [ ]:
# remove punctuations
import string
df['text'] = df['text'].str.replace('[^\w\s]',' ')

In [ ]:
# convert to lower so that all stopwords are removed
# import string
df['text'] = df['text'].str.lower()

In [ ]:
# create word vector (tokenize)
# from nltk.tokenize import word_tokenize
df['articles'] = df.apply(lambda row: word_tokenize(row['text']), axis=1)

In [ ]:
# remove common words (stopwords)
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df['articles'] = df['articles'].apply(lambda x: [ y for y in x if y not in stop_words])

In [ ]:
# stem words (stemming)
# from nltk.stem.porter import PorterStemmer
df['stemmed'] = df['articles'].apply(lambda x: [PorterStemmer().stem(y) for y in x])

# letter 'e' is cut from many words from stemming, so we lemmatize instead to get original word
#from nltk.stem import WordNetLemmatizer
#df['stemmed'] = df['articles'].apply(lambda x: [WordNetLemmatizer().lemmatize(y) for y in x])

In [ ]:
# remove non-word characters
# df['stemmed'] = df['stemmed'].apply(lambda x: [y for y in x if y.isalpha()])

In [ ]:
# covert lists to string so that tfidf works
df['list']=[" ".join(element) for element in df['stemmed'].values]
X = df['list']

In [ ]:
# split dataframe to train-test datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
# add tfidf weight to words
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# df1 = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names())
#print(vectorizer.vocabulary_)
#vectorizer.get_feature_names()

In [ ]:
# see shape of X_train
X_train

In [ ]:
# define the keras model
from keras.models import Sequential
from keras.layers import Dense

# define the sequential model with 3 layers
# input_dim must match X_train columns

model = Sequential()
model.add(Dense(12, input_dim="X_train_cols", activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1)

In [ ]:
# predict classes: 0 or 1
y_pred = model.predict_classes(X_test)

In [ ]:
# see shape of y_test
y_test

In [ ]:
# see shape of y_pred
y_pred

In [ ]:
# print perscision-recall-f1_score metrics
from sklearn.metrics import  precision_score, recall_score, f1_score

print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test,y_pred))

In [ ]:
# print classification_report-confusion_matrix-accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))